In [1]:
%pip install oracledb


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\ALTHOURAYA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\ALTHOURAYA\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
Transaction_Fact_insert = '''insert into Transaction_Fact (TransactionID, ProductID, DateID ,CustomerID, CategoryID, SupplierID, OrderID, Quantity, UnitPrice, TotalAmount ) values(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10)'''

In [4]:
import oracledb
import pandas as pd

# Connect to the Oracle database
operational_connection = oracledb.connect(user="walmart", password="walmart", dsn="127.0.0.1:1521/xepdb1")
print("Connected to Oracle Database")
 
# Create a cursor to execute queries
operational_cursor = operational_connection.cursor()
def get_data_frame(cursor, query):
    # Execute the SQL query
    cursor.execute(query)
    # Fetch all rows from the query result
    data = cursor.fetchall()
    # Get column names (metadata)
    columns = [desc[0] for desc in cursor.description]
    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    return df

Connected to Oracle Database


In [5]:
query='''SELECT c.CategoryID, p.ProductID, p.SupplierID,
 cu.CUSTOMERID, o.OrderID, od.ORDERDETAILID, o.TotalAmount,
  o.ORDERDATE, od.UNITPRICE, od.Quantity

FROM walmart.Categories c
JOIN walmart.Products p ON c.CategoryID = p.CategoryID
JOIN walmart.Suppliers s ON p.SupplierID = s.SupplierID
JOIN walmart.OrderDetails od ON p.ProductID = od.ProductID
JOIN walmart.Orders o ON od.OrderID = o.OrderID
JOIN walmart.Customers cu ON o.CustomerID = cu.CustomerID'''



In [39]:
fact_table = get_data_frame(operational_cursor, query)

In [40]:
fact_df = get_data_frame(operational_cursor, query)
                         # Connect to the Oracle database
star_connection = oracledb.connect(user="walmart_star_schema", password="walmart", dsn="127.0.0.1:1521/xepdb1")
star_cursor = star_connection.cursor()
print("Connected to Oracle Database")
def insert_data_frame(cursor, query, dataFrame):
    for i, row in dataFrame.iterrows():
        cursor.execute(query, tuple(row))
    cursor.execute('commit')

Connected to Oracle Database


In [41]:
# Load customer_dim data and put into a data frame. 
customer_dim_query = "SELECT  Row_ID, CustomerID FROM walmart_star_schema.CUSTOMER_DIM"
customer_dim_df = get_data_frame(star_cursor, customer_dim_query)


In [42]:

fact_table = pd.merge(fact_table, customer_dim_df[['CUSTOMERID', 'ROW_ID']], 
                      left_on='CUSTOMERID', right_on='CUSTOMERID', how='inner')


In [47]:
print(fact_table.head())

Empty DataFrame
Columns: [CATEGORYID, PRODUCTID, SUPPLIERID, CUSTOMERID, ORDERID, ORDERDETAILID, TOTALAMOUNT, ORDERDATE, UNITPRICE, QUANTITY, ROW_ID]
Index: []


In [49]:
print(customer_dim_df)

Empty DataFrame
Columns: [ROW_ID, CUSTOMERID]
Index: []


insert into Transaction_Fact (TransactionID, ProductID, DateID ,CustomerID, CategoryID, SupplierID, OrderID, Quantity, UnitPrice, TotalAmount ) values(:1, :2, :3, :4, :5, :6, :7, :8, :9, :10)
